# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
from random import gauss as gs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit

import itertools
#from pmdarima import auto_arima

#statsmodels
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [5]:
df = pd.read_csv('data/Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_month.csv')
df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,1996-01-31,1996-02-29,1996-03-31,1996-04-30,1996-05-31,...,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31
0,102001,0,United States,country,NaN,109265.180383,109095.676695,109155.569885,109363.554893,109612.520655,...,334114.619841,339943.862979,346018.233702,352528.818505,356754.830595,359526.260243,359158.102261,358023.786647,358091.378335,358458.225080
1,394913,1,"New York, NY",msa,NY,188625.267307,186907.738371,186522.213709,186206.002046,186097.935571,...,583133.856863,588755.030374,598934.287161,607910.065648,615702.062132,619968.688450,621554.903213,620482.991765,619756.263314,618741.406995
2,753899,2,"Los Angeles, CA",msa,CA,187535.806762,186864.880014,186572.180966,186548.431121,186416.976291,...,892162.981058,906481.839475,935662.296589,955154.827578,960211.478267,937913.189174,934000.527320,902599.962352,897593.360077,904366.608225
3,394463,3,"Chicago, IL",msa,IL,147099.700804,147098.923393,146179.663686,147598.866028,148127.466534,...,294329.692123,298176.812923,303576.875998,308294.433566,313516.107710,314566.424412,313662.039722,312065.281120,311983.462353,312194.106986
4,394514,4,"Dallas, TX",msa,TX,113009.443351,112925.248113,113244.783251,113772.145381,114010.945821,...,359008.229625,369556.124915,375611.161083,390748.944050,398646.529571,400284.009721,395943.534446,390452.276608,389048.558123,391640.199263


In [6]:
df = pd.read_csv('data/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31
0,91940,0,77449,zip,TX,TX,NaN,"Houston-The Woodlands-Sugar Land, TX",Harris County,107184.0,...,257089.0,261757.0,267452.0,273750.0,279865.0,284916.0,288326.0,290250.0,290894.0,291871.0
1,91982,1,77494,zip,TX,TX,NaN,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,223094.0,...,444749.0,454220.0,465824.0,479498.0,490869.0,498347.0,500609.0,498747.0,499289.0,501368.0
2,93144,2,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,90164.0,...,175658.0,177626.0,179773.0,183128.0,186726.0,190164.0,192396.0,194095.0,195477.0,197127.0
3,62080,3,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,186206.0,...,564682.0,564481.0,566396.0,567898.0,570676.0,572891.0,574998.0,575933.0,578691.0,582277.0
4,62093,4,11385,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,275596.0,...,747687.0,750970.0,756670.0,759268.0,765087.0,770360.0,775517.0,778187.0,781268.0,787104.0


In [7]:
df = pd.read_csv('data/zillow_data.csv')
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [ ]:
df.shape

In [ ]:
df[df.Metro == 'New York']['CountyName'].value_counts()

In [ ]:
ny = df[(df.Metro == 'New York')&(df.CountyName.isin(['Kings', 'Richmond', 'Queens', 'Bronx']))]

In [ ]:
ny.shape

# Step 2: Data Preprocessing

In [ ]:
ny.iloc[:, 0:7].isnull().sum()

In [ ]:
ny.iloc[:, 7:].isnull().sum()

In [ ]:
ny = ny.fillna(method = 'bfill', axis = 1)

In [ ]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[7:], format='%Y-%m')
ny.columns = list(ny.columns[:7]) + list(get_datetimes(ny))
ny.head()

In [ ]:
nydf_copy = ny.copy()
nydf_copy['growth'] = nydf_copy['2018-04'] - nydf_copy['2015-04']
nydf_copy['roi'] = nydf_copy['growth']/nydf_copy['2015-04']*100
top_20_growth = nydf_copy.sort_values('roi', ascending=False)[:20]
top_20_growth

In [ ]:
top_20_growth.plot(x = 'RegionName', y = 'roi', kind = 'bar', figsize = (15,6));

In [ ]:
top_10 = top_20_growth.copy()
top_10['growth'] = top_10['2018-04'] - top_10['2011-04']
top_10['roi'] = top_10['growth']/top_10['2011-04']*100
top_10 = top_10.sort_values('roi', ascending=False)[:10]
top_10

In [ ]:
top_10.plot(x = 'RegionName', y = 'roi', kind = 'bar', figsize = (15,6));

In [ ]:
# Plot the top 10 zipcodes by growth in the period 2008-2018
xticks = ['1996-04', '1997-04', '1998-04', '1999-04', '2000-04', '2001-04', '2002-04', '2003-04', '2004-04', '2005-04', '2006-04', '2007-04', '2008-04', '2009-04', '2010-04', '2011-04', '2012-04', '2013-04', '2014-04', '2015-04', '2016-04', '2017-04', '2018-04']
xlabels = ['1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']

plt.figure(figsize=(16,8))
for index in top_20_growth.index:
    sample = top_20_growth.loc[index,'1996-04':'2018-04']
    zipcode = top_20_growth.loc[index]['RegionName']
    plt.plot(sample, label=top_20_growth.loc[index]['RegionName'])
plt.xticks(xticks, xlabels)
plt.legend(loc='upper left', ncol=2)
plt.title('Top 10 Zipcodes by Growth over Period 2008-2018')
plt.ylabel('Median home price ($)')
plt.xlabel('Year')
plt.show();

In [ ]:
# Plot the top 10 zipcodes by growth in the period 2008-2018
xticks = ['1996-04', '1997-04', '1998-04', '1999-04', '2000-04', '2001-04', '2002-04', '2003-04', '2004-04', '2005-04', '2006-04', '2007-04', '2008-04', '2009-04', '2010-04', '2011-04', '2012-04', '2013-04', '2014-04', '2015-04', '2016-04', '2017-04', '2018-04']
xlabels = ['1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']

plt.figure(figsize=(16,8))
plt.plot(ny_mean)
plt.xticks(xticks, xlabels)
plt.title('Top 10 Zipcodes by Growth over Period 2008-2018')
plt.ylabel('Median home price ($)')
plt.xlabel('Year')
plt.show();

In [ ]:
(ny_mean['2018-04'] - ny_mean['2011-04']) / ny_mean['2011-04']

In [ ]:
(ny_mean['2011-04'] - ny_mean['2007-04']) / ny_mean['2007-04']

In [ ]:
ny_mean['2018-04']

In [ ]:
ny_mean = ny.iloc[:, 7:].mean()

In [ ]:
def get_datetimes(df):
    """
    Takes a dataframe:
    returns only those column names that can be converted into datetime objects 
    as datetime objects.
    NOTE number of returned columns may not match total number of columns in passed dataframe
    """
    
    return pd.to_datetime(df.columns.values[7:], format='%Y-%m')

In [ ]:
get_datetimes(ny)

In [ ]:
ts_11215 = ny[ny.RegionName == 11215]

In [ ]:
ts11215 = get_datetimes(ts_11215)

In [ ]:
ts11215

# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 4: Reshape from Wide to Long Format

In [ ]:
def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [ ]:
ts11215 = melt_data(ts_11215)

In [ ]:
ts = ts11215['2011':'2017-04-01']

In [ ]:
ts.plot()

In [ ]:
from statsmodels.tsa.stattools import adfuller

def stationarity_check(data):
    roll_mean = data.rolling(window = 6, center = False).mean()
    roll_std = data.rolling(window = 6, center = False).std()
    fig = plt.figure(figsize=(12,7))
    plt.plot(data, color='blue', label='Original')
    plt.plot(roll_mean, color='red', label='Rolling Mean')
    plt.plot(roll_std, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    dftest = adfuller(data)
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    return dfoutput

In [ ]:
stationarity_check(ts)

In [ ]:
stationarity_check(tsone.diff().dropna())

In [ ]:
tsone = ts.diff(periods = 1).dropna()

In [ ]:
tstwo = tsone.diff().dropna()

In [ ]:
stationarity_check(tsone.diff(periods=6).dropna())

# Step 5: ARIMA Modeling

In [ ]:
plot_pacf(tsone, lags = 30, method = 'ywm');

In [ ]:
plot_acf(tsone, lags = 30);

In [ ]:
plot_pacf(tstwo, lags = 30, method = 'ywm');

In [ ]:
plot_acf(tstwo, lags = 30);

# Step 6: Interpreting Results

In [ ]:
%pip install pmdarima

In [ ]:
from pmdarima import auto_arima

In [ ]:
model = auto_arima(ts, start_p=0, start_q=0,
                             max_p=2, max_q=2, m=12,
                             start_P=0, start_Q = 0, seasonal=True,
                            trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True)  # set to stepwise

                   
model.fit(ts)

In [ ]:
model.summary()

In [ ]:
model.plot_diagnostics(figsize=(14,10));

In [ ]:
prediction, confint = model.predict(n_periods=12, return_conf_int=True)

prediction

In [ ]:
confint

In [ ]:
prediction.plot()

In [ ]:
from pmdarima.arima.utils import ndiffs

In [ ]:
from pmdarima.arima.utils import nsdiffs

In [ ]:
ndiffs(ts, test='adf', alpha = 0.05)

In [ ]:
nsdiffs(ts, m= 12, test ='ocsb')

In [ ]:
stepwise_fit = auto_arima(ts, start_p=1, start_q=1,
                             max_p=3, max_q=3, m=12,
                             start_P=0, seasonal=True,
                             D = 1, d = 1,
                             trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True)  # set to stepwise

In [ ]:
stepwise_fit.summary()

In [ ]:
resid = stepwise_fit.resid()

In [ ]:
plot_pacf(resid, lags = 25, method = 'ywm');

In [ ]:
plot_acf(resid, lags = 25);

In [ ]:
smodel = SARIMAX(ts, order = (2,1,1), seasonal_order = (0,1,0,12)).fit()

In [ ]:
smodel.summary()

In [ ]:
smodel.plot_diagnostics(figsize = (15,6));

In [ ]:
fc = smodel.forecast(36)

In [ ]:
ts11215['2017-04-01':].plot()
fc.plot()

In [ ]:
smodel.predict().plot()

In [ ]:
pred = smodel.get_prediction(start=pd.to_datetime('2011-06-01'), dynamic=False)
pred.predicted_mean
# Plot observed values
ts11215['2011':].plot()

# Plot predicted values
pred.predicted_mean.plot()

In [ ]:
smodel.predict().head(20)

In [ ]:
ts11215['2012']